In [1]:
import pandas as pd 
import numpy as np
import csv
import advance_functions as advance
import RiskSLIM as slim
import stumps
from sklearn.linear_model import LogisticRegression
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_F6_ky_inter_model

no stored variable summary_F6_ky_inter_model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_F_six_month'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_F_six_month'].values

In [3]:
#### CART
depth = [7,8,9]
cart_summary = advance.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [20]
depth = [2]
learning_rate = [0.5]
ebm_summary = advance.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [4]:
cart_summary, ebm_summary

({'best_auc': 0.7640925540244371,
  'auc_diff': 0.017240677082636657,
  'best_param': {'max_depth': 7},
  'FL_score': 0.5771567867751518},
 {'best_auc': 0.7627011686257845,
  'auc_diff': 0.009499760470505825,
  'best_param': {'learning_rate': 0.5,
   'max_tree_splits': 2,
   'n_estimators': 20},
  'FL_score': 0.5241379310344828})

### Lasso Stumps

In [9]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence>=1']
KY_Y = KY_stumps['recid_F_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence>=1']
FL_Y = FL_stumps['recid_F_six_month'].values

## columns 
cols = KY_X.columns

In [10]:
c_grid = {'C':[0.03, 0.05, 0.07]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [11]:
stumps_summary

{'best_auc': 0.7662315937610928,
 'best_params': {'C': 0.05},
 'auc_diffs': 0.004370012455411798,
 'FL_score': 0.5525393904171509}

### RiskSLIM

In [12]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.05, random_state=816).fit(KY_X, KY_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

71

In [14]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [15]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'general2', 
                                max_coef = 20, 
                                max_coef_number = 10, 
                                max_runtime=200, 
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
09/17/19 @ 08:37 PM | 2941 rows in lookup table
09/17/19 @ 08:37 PM | ------------------------------------------------------------
09/17/19 @ 08:37 PM | runnning initialization procedure
09/17/19 @ 08:37 PM | ------------------------------------------------------------
09/17/19 @ 08:37 PM | CPA produced 2 cuts
09/17/19 @ 08:37 PM | running naive rounding on 88 solutions
09/17/19 @ 08:37 PM | best objective value: 0.1924
09/17/19 @ 08:37 PM | rounding produced 5 integer solutions
09/17/19 @ 08:37 PM | best objective value is 0.1975
09/17/19 @ 08:37 PM | running sequential rounding on 88 solutions
09/17/19 @ 08:37 PM | best objective value: 0.1924
09/17/19 @ 08:37 PM | sequential rounding produced 5 integer solutions
09/17/19 @ 08:37 PM | best objective value: 0.1975
09/17/19 @ 08:37 PM | polishing 10 solutions
09/17/19 @ 08:37 PM | best objective value: 0.1975
09/17/19 @ 08:37 PM | polishing produced 5 integer solutions
09/17/19

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
09/17/19 @ 08:37 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1975.
Tried aggregator 1 time.
Reduced MIP has 73 rows, 146 columns, and 287 nonzeros.
Reduced MIP has 71 binaries, 73 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.17 ticks)
Probing time = 0.00 sec. (0.08 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.09 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Varia

In [16]:
riskslim_summary

{'FL_Score': 0.5511494252873563}

In [17]:
#### save results
summary_F6_ky_inter_model = {"CART": cart_summary,
                                    "EBM": ebm_summary,
                                    "Stumps": stumps_summary,
                                    "RiskSLIM": riskslim_summary}
%store summary_F6_ky_inter_model

Stored 'summary_F6_ky_inter_model' (dict)


In [19]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Six Month\\"
results = [["Felony",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_Score'], 3)]]
with open(path + 'Six Month KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)